In [1]:
# =============================
# library
# =============================
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import polars as pl
from tqdm import tqdm
import pickle

In [2]:
# =============================
# constant
# =============================
#TRAIN_PATH = Path("../storage/leap/data/train.csv")
#TEST_PATH = Path("../storage/leap/data/test.csv")
OUTPUT_DIR = Path("../storage/leap/output")
DATA_DIR = Path("../storage/leap/data")

In [3]:
# =============================
# settings
# =============================
fe = "177"
fe_dir = OUTPUT_DIR / "fe" / f"fe{fe}"
fe_dir.mkdir(parents=True, exist_ok=True)
fe_save_dir = fe_dir / "save"
fe_save_dir.mkdir(parents=True, exist_ok=True)

TEST_PATH1 = Path("../storage/leap/output/fe/fe100/fe100_test.parquet")
TEST_OLD_PATH1 = Path("../storage/leap/output/fe/fe173/fe173_test.parquet")
train_state_sum_sc_path = OUTPUT_DIR / "fe" / "fe103" / "fe103_state_sum_mean_std.pkl"

In [4]:
# =============================
# main
# =============================
df = pl.read_parquet(TEST_PATH1)
df_old = pl.read_parquet(TEST_OLD_PATH1)

In [5]:
df = df[['sample_id',
 'state_t_0',
 'state_t_1',
 'state_t_2',
 'state_t_3',
 'state_t_4',
 'state_t_5',
 'state_t_6',
 'state_t_7',
 'state_t_8',
 'state_t_9',
 'state_t_10',
 'state_t_11',
 'state_t_12',
 'state_t_13',
 'state_t_14',
 'state_t_15',
 'state_t_16',
 'state_t_17',
 'state_t_18',
 'state_t_19',
 'state_t_20']]
df_old = df_old[['sample_id',
 'state_t_0',
 'state_t_1',
 'state_t_2',
 'state_t_3',
 'state_t_4',
 'state_t_5',
 'state_t_6',
 'state_t_7',
 'state_t_8',
 'state_t_9',
 'state_t_10',
 'state_t_11',
 'state_t_12',
 'state_t_13',
 'state_t_14',
 'state_t_15',
 'state_t_16',
 'state_t_17',
 'state_t_18',
 'state_t_19',
 'state_t_20']]

In [6]:
df = df.with_columns([pl.col(c).cast(pl.Float32) for c in df.columns[1:]])
df_old = df_old.with_columns([pl.col(c).cast(pl.Float32) for c in df_old.columns[1:]])

In [7]:
df = df.rename({'sample_id': 'new_sample_id'})

In [8]:
df_old = df_old.join(df,how="left",on=['state_t_0',
 'state_t_1',
 'state_t_2',
 'state_t_3',
 'state_t_4',
 'state_t_5',
 'state_t_6',
 'state_t_7',
 'state_t_8',
 'state_t_9',
 'state_t_10',
 'state_t_11',
 'state_t_12',
 'state_t_13',
 'state_t_14',
 'state_t_15',
 'state_t_16',
 'state_t_17',
 'state_t_18',
 'state_t_19',
 'state_t_20'])

In [9]:
df_old = df_old.to_pandas()

In [10]:
df_old["num"] = np.arange(len(df_old))
df_old["time"] = df_old["num"] // 384

In [11]:
df_old[df_old["new_sample_id"].notnull()]["time"].value_counts().value_counts()

count
384    326
Name: count, dtype: int64

In [12]:
df_old = df_old[["sample_id","new_sample_id","time"]].reset_index(drop=True)

In [13]:
df_old.to_parquet(fe_dir / "test_old_test_match.parquet")